# Configuração

Importar biliotecas necessárias

In [111]:
import pandas as pd
import boto3
import numpy as np
from sklearn.datasets import dump_svmlight_file 
import sagemaker

Especificação do Bucket S3 e identificação da função de execução (role)

In [112]:
bucket = 'datascience-sagemaker-fernandosousa'

# Dados

Leitura dos dados do S3

In [113]:
dados_brutos_arquivo = 'Case_cobranca.csv'

s3 = boto3.resource('s3')
s3.Bucket(bucket).download_file(dados_brutos_arquivo, 'dados_brutos.csv')

dados = pd.read_csv('./dados_brutos.csv')
pd.set_option('display.max_rows', 20) 
dados

,COD,TIPO_CLIENTE,TIPO_EMPRESTIMO,CD_SEXO,IDADE,QTD_DIVIDAS,TEMP_RECUPERACAO
0,1,NOVO,CDC,M,26.0,0,227.0
1,2,NOVO,RENEGOCIAÇÃO,M,39.0,0,37.0
2,3,NOVO,PESSOAL,M,31.0,1,NaN
3,4,NOVO,PESSOAL,F,45.0,11,NaN
4,5,NOVO,CDC,F,25.0,0,8.0
...,...,...,...,...,...,...,...
9812,9813,NOVO,CDC,F,38.0,1,23.0
9813,9814,NOVO,RENEGOCIAÇÃO,M,34.0,2,234.0
9814,9815,TOMADOR,CDC,NaN,30.0,4,13.0
9815,9816,INVESTIDOR,CDC,M,43.0,0,6.0


# Preparação dos dados

Coluna ALVO.

Variável dependente, objetivo do algoritmo de classificação

Bons pagadores: tempo de recuperação até 90 dias (1)

Maus pagadores: tempo de recuperação mais de 90 dias (0)

In [114]:
dados['ALVO']   = dados['TEMP_RECUPERACAO'].apply(lambda x: 1 if x <= 90 else 0)
dados


,COD,TIPO_CLIENTE,TIPO_EMPRESTIMO,CD_SEXO,IDADE,QTD_DIVIDAS,TEMP_RECUPERACAO,ALVO
0,1,NOVO,CDC,M,26.0,0,227.0,0
1,2,NOVO,RENEGOCIAÇÃO,M,39.0,0,37.0,1
2,3,NOVO,PESSOAL,M,31.0,1,NaN,0
3,4,NOVO,PESSOAL,F,45.0,11,NaN,0
4,5,NOVO,CDC,F,25.0,0,8.0,1
...,...,...,...,...,...,...,...,...
9812,9813,NOVO,CDC,F,38.0,1,23.0,1
9813,9814,NOVO,RENEGOCIAÇÃO,M,34.0,2,234.0,0
9814,9815,TOMADOR,CDC,NaN,30.0,4,13.0,1
9815,9816,INVESTIDOR,CDC,M,43.0,0,6.0,1


Codificação TIPO_CLIENTE

In [115]:
dados['CLIENTE_NOVO']   = dados['TIPO_CLIENTE'].apply(lambda x: 1 if x == 'NOVO' else 0)
dados['CLIENTE_INVESTIDOR']   = dados['TIPO_CLIENTE'].apply(lambda x: 1 if x == 'INVESTIDOR' else 0)             

dados

,COD,TIPO_CLIENTE,TIPO_EMPRESTIMO,CD_SEXO,IDADE,QTD_DIVIDAS,TEMP_RECUPERACAO,ALVO,CLIENTE_NOVO,CLIENTE_INVESTIDOR
0,1,NOVO,CDC,M,26.0,0,227.0,0,1,0
1,2,NOVO,RENEGOCIAÇÃO,M,39.0,0,37.0,1,1,0
2,3,NOVO,PESSOAL,M,31.0,1,NaN,0,1,0
3,4,NOVO,PESSOAL,F,45.0,11,NaN,0,1,0
4,5,NOVO,CDC,F,25.0,0,8.0,1,1,0
...,...,...,...,...,...,...,...,...,...,...
9812,9813,NOVO,CDC,F,38.0,1,23.0,1,1,0
9813,9814,NOVO,RENEGOCIAÇÃO,M,34.0,2,234.0,0,1,0
9814,9815,TOMADOR,CDC,NaN,30.0,4,13.0,1,0,0
9815,9816,INVESTIDOR,CDC,M,43.0,0,6.0,1,0,1


Codificação TIPO_EMPRESTIMO

In [116]:
dados['EMPRESTIMO_CDC']   = dados['TIPO_EMPRESTIMO'].apply(lambda x: 1 if x == 'CDC' else 0)
dados['EMPRESTIMO_PESSOAL']   = dados['TIPO_EMPRESTIMO'].apply(lambda x: 1 if x == 'PESSOAL' else 0)
dados

,COD,TIPO_CLIENTE,TIPO_EMPRESTIMO,CD_SEXO,IDADE,QTD_DIVIDAS,TEMP_RECUPERACAO,ALVO,CLIENTE_NOVO,CLIENTE_INVESTIDOR,EMPRESTIMO_CDC,EMPRESTIMO_PESSOAL
0,1,NOVO,CDC,M,26.0,0,227.0,0,1,0,1,0
1,2,NOVO,RENEGOCIAÇÃO,M,39.0,0,37.0,1,1,0,0,0
2,3,NOVO,PESSOAL,M,31.0,1,NaN,0,1,0,0,1
3,4,NOVO,PESSOAL,F,45.0,11,NaN,0,1,0,0,1
4,5,NOVO,CDC,F,25.0,0,8.0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9812,9813,NOVO,CDC,F,38.0,1,23.0,1,1,0,1,0
9813,9814,NOVO,RENEGOCIAÇÃO,M,34.0,2,234.0,0,1,0,0,0
9814,9815,TOMADOR,CDC,NaN,30.0,4,13.0,1,0,0,1,0
9815,9816,INVESTIDOR,CDC,M,43.0,0,6.0,1,0,1,1,0


Codificação CD_SEXO

In [117]:
dados['SEXO_M']   = dados['CD_SEXO'].apply(lambda x: 1 if x == 'M' else 0)
dados

,COD,TIPO_CLIENTE,TIPO_EMPRESTIMO,CD_SEXO,IDADE,QTD_DIVIDAS,TEMP_RECUPERACAO,ALVO,CLIENTE_NOVO,CLIENTE_INVESTIDOR,EMPRESTIMO_CDC,EMPRESTIMO_PESSOAL,SEXO_M
0,1,NOVO,CDC,M,26.0,0,227.0,0,1,0,1,0,1
1,2,NOVO,RENEGOCIAÇÃO,M,39.0,0,37.0,1,1,0,0,0,1
2,3,NOVO,PESSOAL,M,31.0,1,NaN,0,1,0,0,1,1
3,4,NOVO,PESSOAL,F,45.0,11,NaN,0,1,0,0,1,0
4,5,NOVO,CDC,F,25.0,0,8.0,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9812,9813,NOVO,CDC,F,38.0,1,23.0,1,1,0,1,0,0
9813,9814,NOVO,RENEGOCIAÇÃO,M,34.0,2,234.0,0,1,0,0,0,1
9814,9815,TOMADOR,CDC,NaN,30.0,4,13.0,1,0,0,1,0,0
9815,9816,INVESTIDOR,CDC,M,43.0,0,6.0,1,0,1,1,0,1


Normalização IDADE

In [118]:
dados['IDADE_NORM'] = dados['IDADE'].apply(lambda x: 18 if np.isnan(x) or x < 18 else x) # mínimo
dados['IDADE_NORM'] = dados['IDADE_NORM'].apply(lambda x: 76 if x > 76 else x) # máximo
dados['IDADE_NORM'] = dados['IDADE_NORM'].apply(lambda x: (x-18)/(76-18)) # normalização entre 0 e 1
dados


,COD,TIPO_CLIENTE,TIPO_EMPRESTIMO,CD_SEXO,IDADE,QTD_DIVIDAS,TEMP_RECUPERACAO,ALVO,CLIENTE_NOVO,CLIENTE_INVESTIDOR,EMPRESTIMO_CDC,EMPRESTIMO_PESSOAL,SEXO_M,IDADE_NORM
0,1,NOVO,CDC,M,26.0,0,227.0,0,1,0,1,0,1,0.137931
1,2,NOVO,RENEGOCIAÇÃO,M,39.0,0,37.0,1,1,0,0,0,1,0.362069
2,3,NOVO,PESSOAL,M,31.0,1,NaN,0,1,0,0,1,1,0.224138
3,4,NOVO,PESSOAL,F,45.0,11,NaN,0,1,0,0,1,0,0.465517
4,5,NOVO,CDC,F,25.0,0,8.0,1,1,0,1,0,0,0.120690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9812,9813,NOVO,CDC,F,38.0,1,23.0,1,1,0,1,0,0,0.344828
9813,9814,NOVO,RENEGOCIAÇÃO,M,34.0,2,234.0,0,1,0,0,0,1,0.275862
9814,9815,TOMADOR,CDC,NaN,30.0,4,13.0,1,0,0,1,0,0,0.206897
9815,9816,INVESTIDOR,CDC,M,43.0,0,6.0,1,0,1,1,0,1,0.431034


Normalização QTD_DIVIDAS

In [119]:
dados['QTD_DIVIDAS_NORM'] = dados['QTD_DIVIDAS'].apply(lambda x: 0. if np.isnan(x) else x/16) # normalização entre 0 e 1
dados

,COD,TIPO_CLIENTE,TIPO_EMPRESTIMO,CD_SEXO,IDADE,QTD_DIVIDAS,TEMP_RECUPERACAO,ALVO,CLIENTE_NOVO,CLIENTE_INVESTIDOR,EMPRESTIMO_CDC,EMPRESTIMO_PESSOAL,SEXO_M,IDADE_NORM,QTD_DIVIDAS_NORM
0,1,NOVO,CDC,M,26.0,0,227.0,0,1,0,1,0,1,0.137931,0.0000
1,2,NOVO,RENEGOCIAÇÃO,M,39.0,0,37.0,1,1,0,0,0,1,0.362069,0.0000
2,3,NOVO,PESSOAL,M,31.0,1,NaN,0,1,0,0,1,1,0.224138,0.0625
3,4,NOVO,PESSOAL,F,45.0,11,NaN,0,1,0,0,1,0,0.465517,0.6875
4,5,NOVO,CDC,F,25.0,0,8.0,1,1,0,1,0,0,0.120690,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9812,9813,NOVO,CDC,F,38.0,1,23.0,1,1,0,1,0,0,0.344828,0.0625
9813,9814,NOVO,RENEGOCIAÇÃO,M,34.0,2,234.0,0,1,0,0,0,1,0.275862,0.1250
9814,9815,TOMADOR,CDC,NaN,30.0,4,13.0,1,0,0,1,0,0,0.206897,0.2500
9815,9816,INVESTIDOR,CDC,M,43.0,0,6.0,1,0,1,1,0,1,0.431034,0.0000


Remover colunas desnecessárias

In [120]:
dados = dados.drop(['COD', 'TIPO_CLIENTE', 'TIPO_EMPRESTIMO', 'CD_SEXO', 'IDADE', 'QTD_DIVIDAS', 'TEMP_RECUPERACAO'], axis=1)
dados

,ALVO,CLIENTE_NOVO,CLIENTE_INVESTIDOR,EMPRESTIMO_CDC,EMPRESTIMO_PESSOAL,SEXO_M,IDADE_NORM,QTD_DIVIDAS_NORM
0,0,1,0,1,0,1,0.137931,0.0000
1,1,1,0,0,0,1,0.362069,0.0000
2,0,1,0,0,1,1,0.224138,0.0625
3,0,1,0,0,1,0,0.465517,0.6875
4,1,1,0,1,0,0,0.120690,0.0000
...,...,...,...,...,...,...,...,...
9812,1,1,0,1,0,0,0.344828,0.0625
9813,0,1,0,0,0,1,0.275862,0.1250
9814,1,0,0,1,0,0,0.206897,0.2500
9815,1,0,1,1,0,1,0.431034,0.0000


Separação em treino (70%), validação(20%) e testes (10%)

In [121]:
treinamento, validacao, teste = np.split(dados.sample(frac=1, random_state=88), [int(0.7 * len(dados)), int(0.9 * len(dados))]) 
len(dados),len(treinamento), len(validacao), len(teste)

(9817, 6871, 1964, 982)

Transformação para formato do algoritmo XGBoost

https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

In [122]:
treinamento.to_csv('treinamento.csv', index=False, header=False)
validacao.to_csv('validacao.csv', index=False, header=False)
teste.to_csv('teste.csv', index=False)

Salvar no S3

In [123]:
boto3.Session().resource('s3').Bucket(bucket).Object('treinamento.csv').upload_file('treinamento.csv')
boto3.Session().resource('s3').Bucket(bucket).Object('validacao.csv').upload_file('validacao.csv')
boto3.Session().resource('s3').Bucket(bucket).Object('teste.csv').upload_file('teste.csv')